In [4]:
# -*- coding: utf-8 -*-
from __future__ import division
from sklearn.metrics import accuracy_score

__author__ = 'alex'

import numpy as np
import pandas as pd
import math
from time import clock
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import expit as sigmoid
from scipy.optimize import fmin_l_bfgs_b
from matplotlib import pyplot as plt
from numba import jit
#%matplotlib inline

def plot_digit(idx):
    img = X_train[idx]
    if img.shape[0]>784:
        img = img[:-1]
    img = np.reshape(img, (28,28))
    plt.imshow(img, cmap='Greys',  interpolation='nearest')
    plt.title('true label: %d' % y_train[idx])
    plt.show()

data = np.load('../Data/mnist.npz')

X_train = data['images_train']/255
y_train = data['labels_train']
X_test = data['images_test']/255
y_test = data['labels_test']

def add_bias(x):
    ones = np.ones((x.shape[0],1))
    return np.hstack((x, ones))

def shuffle_data(X,y):
    idx = np.random.permutation(len(X))
    return X[idx], y[idx]

def zscore(X):
    return (X - X.mean(axis=1).reshape(len(X),1))/X.std(axis=1).reshape(len(X),1)

def PCA(X):
    pass
    #PCA - leave 185 dimensions

X_train, y_train = shuffle_data(X_train, y_train)
X_train = add_bias(X_train)
X_test = add_bias(X_test)

X_train = zscore(X_train)
X_test = zscore(X_test)

n_classes = len(set(y_train))


class LogisticRegression():
    def __init__(self, dim):
        self.w = np.zeros(dim)
        self.eta = 10e-7

    def Gw(self, x):
        return sigmoid(x.dot(self.w))

    def cost(self, y_true, y_pred):
        return -(y_true * np.ma.log(y_pred).data + (1-y_true) * np.ma.log(1-y_pred).data).sum()

    def gradient(self, y_true, y_pred, X):
        return -X * (y_true - y_pred).reshape(y_true.shape[0],1)

    def SGD(self, y_true, y_pred, X):
        self.w = self.w - self.eta * self.gradient(y_true,y_pred,X).sum(axis=0)

    def fit(self, X_train, y_train, mini_batch = 100, verbose = 10):
        old_loss = np.inf
        of = 0
        for i in range(15):
            X_train, y_train = shuffle_data(X_train, y_train)
            for b in range(mini_batch, X_train.shape[0], mini_batch):
                x_batch, y_batch = X_train[b-mini_batch:b], y_train[b-mini_batch:b]
                y_pred = self.predict(x_batch)
                self.SGD(y_batch, y_pred, x_batch)

            if i % verbose == 0:
                loss = self.cost(y_batch, y_pred)
                if old_loss - loss >= 0:
                    of -= 1
                    self.eta *= 2
                else:
                    of += 1
                    self.eta /= 3
                    if of > 5:
                        self.eta /= 20
                        if of > 10:
                            break
                print 'rate: ', self.eta
                old_loss = loss

                print 'loss: ', loss
            if loss < 0.001:
                print 'enough (loss = 0.01)'
                break

    def predict(self, X):
        return self.Gw(X)

    def score(self, X, y_true):
        y_pred = (self.predict(X) >= 0.5).astype(int)
        #p = pd.DataFrame(y_pred.T)
        #p[1] = y_train.T
        #p[:100].plot()
        return accuracy_score(y_true, y_pred)

class SoftmaxRegression(LogisticRegression):
    def __init__(self, dim):
        LogisticRegression.__init__(self, dim)

    def Gw(self, x):
        return np.exp(x.dot(self.w))/np.exp(x.dot(self.w)).sum(axis=1).reshape((100,1))

    def cost(self, y_true, y_pred):
        return -(y_true * np.ma.log(y_pred).data).sum().sum()

    def SGD(self, y_true, y_pred, X):
        self.w = self.w - self.eta * self.gradient(y_true,y_pred,X).sum(axis=1).T

    def gradient(self, y_true, y_pred, X):
        return -X.reshape((1, X.shape[0], X.shape[1])).repeat(10,0) * (y_true - y_pred).T.reshape(y_true.shape[1],y_true.shape[0],1)

    def score(self, X, y_true):
        y_pred = self.predict(X)
        return accuracy_score(y_true, y_pred)
    
@jit
def run_lr():
    dim = X_train.shape[1]

    for i in range(10):
        lr = LogisticRegression(dim)
        y_true = (y_train == i).astype(int)

        lr.fit(X_train, y_true)

        y_true_test = (y_test == i).astype(int)
        print 'test accuracy: ', lr.score(X_test, y_true_test)

In [5]:
t1 = clock()
from sklearn.preprocessing import LabelBinarizer

dim = (X_train.shape[1], n_classes)
lb = LabelBinarizer()
y_true = lb.fit_transform(y_train)
y_true_test = lb.fit_transform(y_test)

sr = SoftmaxRegression(dim)
sr.fit(X_train, y_true, verbose=1)
print 'test accuracy: ', sr.score(X_test, y_true_test)
print 'total time: ', clock()-t1

rate:  2e-06
loss:  175.01295073
rate:  4e-06
loss:  119.881747458
rate:  8e-06
loss:  88.736968331
rate:  1.6e-05
loss:  52.6024796832
rate:  3.2e-05
loss:  50.8145692304
rate:  6.4e-05
loss:  35.540632843
rate:  0.000128
loss:  25.9173380977
rate:  0.000256
loss:  21.0566559244
rate:  8.53333333333e-05
loss:  40.9875989078
rate:  0.000170666666667
loss:  22.2383593884
rate:  5.68888888889e-05
loss:  34.4617850756
rate:  0.000113777777778
loss:  29.3151847278
rate:  0.000227555555556
loss:  24.7824094775
rate:  7.58518518519e-05
loss:  30.5033038459
rate:  0.000151703703704
loss:  26.0846219414
test accuracy: 

ValueError: total size of new array must be unchanged

In [ ]:
print 'test accuracy: ', sr.score(X_test, y_true_test)

In [13]:
import numpy as np
def test1():
    a = np.ones((100,100))
    b = a
    c = b
    d = c
    return d

def test2():
    return np.ones((100,100))

%timeit test1()
%timeit test2()

The slowest run took 18.55 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 3.98 µs per loop
100000 loops, best of 3: 3.98 µs per loop
